# Settings

In [1]:
DEBUG = False

In [2]:
from pylfit.objects import Rule
from pylfit.models import DMVLP
from pylfit.datasets import DiscreteStateTransitionsDataset
from pylfit.postprocessing import compute_counterfactuals

In [3]:
# Handmade program
features = [("a",["0","1"]), ("b",["0","1"]), ("c",["0","1","2"])] 
targets = [("y",["0","1","2"])]

rules = """
y(0) :- a(0).
y(0) :- b(0).
y(0) :- c(0).

y(1) :- b(1).
y(1) :- a(0), c(1).
y(1) :- c(2).

y(2) :- a(1), b(1), c(2).
"""

model = DMVLP(features, targets)
model.compile()

model.rules = [Rule.from_string(s.strip(), features, targets) for s in rules.strip().split("\n") if len(s) > 0 ]
model.summary()

# Check it is complete
if DEBUG:
    data = []
    for s1 in model.feature_states():
        transitions = model.predict([s1])
        for s2 in transitions[tuple(s1)]:
            data.append((s1, s2))

    print(data)

    dataset = DiscreteStateTransitionsDataset(data, features, targets)
    dataset.summary()

    model = DMVLP(features, targets)
    model.compile()
    model.fit(dataset)
    model.summary()

# Counterfactuals

state = ["0","1","1"]

compute_counterfactuals(model, state, "y", ["1"], ["0","2"], verbose=0)

DMVLP summary:
 Algorithm: gula
 Features: 
  a: ['0', '1']
  b: ['0', '1']
  c: ['0', '1', '2']
 Targets: 
  y: ['0', '1', '2']
 Rules:
  y(0) :- a(0).
  y(0) :- b(0).
  y(0) :- c(0).
  y(1) :- b(1).
  y(1) :- a(0), c(1).
  y(1) :- c(2).
  y(2) :- a(1), b(1), c(2).


{'0': [{b(0), c(0)}, {a(1), b(0)}], '2': []}